In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import re
import random
import gc
import os
from tqdm import tqdm

# --- CONFIGURATION ---
INPUT_FILE = "robust_experience_training_data_unbalanced.csv" # Your original source
MAX_LEN = 128
BATCH_SIZE = 4
ACCUMULATION_STEPS = 8  # Effective Batch Size = 32 (4 * 8) -> Much more stable
EPOCHS = 6              # Lower epochs to prevent overfitting
LR = 2e-5
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- 1. ROBUST TEXT CLEANING ---
def clean_text(text):
    text = str(text).lower()
    # Remove dates (e.g., Jan 2020, 2019-2022)
    text = re.sub(r'\b(jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)[a-z]*\.? \d{4}', '', text)
    text = re.sub(r'\d{4}\s?-\s?(present|current|\d{4})', '', text)
    # Remove locations (simple heuristic for common resume formats)
    text = re.sub(r'[a-z]+, [a-z]{2} \|', '', text) 
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# --- 2. GENERATE INTELLIGENT ADVERSARIAL DATA ---
# This creates specific "Hard Negatives" to force the model to learn context.
def generate_adversarial_data():
    data = []

    # A. TEACH: "Consultant" can be Full-time
    ft_consultant_templates = [
        "Full-time Implementation Consultant at Oracle. I work permanently for the firm.",
        "Senior Strategy Consultant (Internal Employee). Not a contractor.",
        "Staff Consultant. Led the client project as a full-time employee.",
        "Permanent Solutions Consultant. Salaried role with benefits.",
        "Principal Consultant. I was employed full-time to manage key accounts."
    ]
    # Replicate these heavily to force the model to learn
    for _ in range(200): 
        data.append({'text': random.choice(ft_consultant_templates), 'label': 'Full-time'})

    # B. TEACH: "Managing Interns" is Full-time, not Internship
    ft_manager_templates = [
        "Engineering Lead. My responsibilities included mentoring summer interns.",
        "Senior Manager. I managed a team of 3 interns and 2 juniors.",
        "Program Coordinator. I organized the internship orientation.",
        "Director of Operations. Oversaw the summer analyst internship program.",
        "Team Lead. Conducted code reviews for the intern team."
    ]
    for _ in range(200):
        data.append({'text': random.choice(ft_manager_templates), 'label': 'Full-time'})

    # C. TEACH: "Intern" means you are learning, not working full-time
    intern_templates = [
        "Marketing Intern. Assisted the manager with social media.",
        "Summer Analyst. Rotational program in finance and risk.",
        "Engineering Intern. Shadowed senior developers and fixed bugs.",
        "Co-op Student. 4-month placement during university.",
        "Virtual Intern. Completed simulated tasks for the company."
    ]
    for _ in range(200):
        data.append({'text': random.choice(intern_templates), 'label': 'Internship'})

    # D. TEACH: "Contractor/Freelance" vs Full-time
    freelance_templates = [
        "Independent Contractor. Worked on a per-project basis.",
        "Freelance Consultant. Advised multiple clients simultaneously.",
        "Self-employed Developer. Built websites for local businesses.",
        "Gig Worker. Completed tasks via a mobile platform.",
        "Temporary Staff. 3-month contract to cover seasonal demand."
    ]
    for _ in range(200):
        data.append({'text': random.choice(freelance_templates), 'label': 'Freelance'})

    return pd.DataFrame(data)

# --- 3. LOAD AND PREPARE DATA ---
if not os.path.exists(INPUT_FILE):
    print(f"Error: {INPUT_FILE} not found. Please ensure your base CSV is present.")
else:
    print("Loading and cleaning data...")
    df = pd.read_csv(INPUT_FILE)
    
    # Clean original data
    df['text'] = df['text'].apply(clean_text)
    
    # Generate and append the "Smart Poison"
    df_poison = generate_adversarial_data()
    df = pd.concat([df, df_poison], ignore_index=True)
    
    # Shuffle
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)

    print(f"Training Data Size after injection: {len(df)}")

    # Encode Labels
    le = LabelEncoder()
    df['label_encoded'] = le.fit_transform(df['label'])
    classes = le.classes_

    # Split
    X_train, X_test, y_train, y_test = train_test_split(
        df['text'].values, df['label_encoded'].values, test_size=0.1, random_state=42, stratify=df['label_encoded']
    )

    # --- 4. DATASET CLASS ---
    class ResumeDataset(Dataset):
        def __init__(self, texts, labels, tokenizer, max_len):
            self.texts = texts
            self.labels = labels
            self.tokenizer = tokenizer
            self.max_len = max_len

        def __len__(self):
            return len(self.texts)

        def __getitem__(self, item):
            text = str(self.texts[item])
            label = self.labels[item]
            encoding = self.tokenizer.encode_plus(
                text, add_special_tokens=True, max_length=self.max_len,
                return_token_type_ids=False, padding='max_length',
                truncation=True, return_attention_mask=True, return_tensors='pt',
            )
            return {
                'input_ids': encoding['input_ids'].flatten(),
                'attention_mask': encoding['attention_mask'].flatten(),
                'labels': torch.tensor(label, dtype=torch.long)
            }

    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    train_dataset = ResumeDataset(X_train, y_train, tokenizer, MAX_LEN)
    test_dataset = ResumeDataset(X_test, y_test, tokenizer, MAX_LEN)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

    # --- 5. ROBUST MODEL SETUP ---
    print("\nInitializing BERT with high Dropout (to prevent overfitting)...")
    config = BertConfig.from_pretrained(
        'bert-base-uncased', 
        num_labels=len(classes),
        hidden_dropout_prob=0.3,          # Increased from 0.1
        attention_probs_dropout_prob=0.3  # Increased from 0.1
    )
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', config=config)
    model = model.to(DEVICE)
    
    optimizer = AdamW(model.parameters(), lr=LR)
    
    # GradScaler for FP16
    scaler = torch.cuda.amp.GradScaler()

    # --- 6. TRAINING LOOP WITH ACCUMULATION ---
    print("\nStarting Training...")
    
    for epoch in range(EPOCHS):
        model.train()
        total_loss = 0
        loop = tqdm(train_loader, leave=True)
        
        optimizer.zero_grad()
        
        for i, batch in enumerate(loop):
            input_ids = batch['input_ids'].to(DEVICE)
            mask = batch['attention_mask'].to(DEVICE)
            labels = batch['labels'].to(DEVICE)

            with torch.cuda.amp.autocast():
                outputs = model(input_ids, attention_mask=mask, labels=labels)
                loss = outputs.loss / ACCUMULATION_STEPS # Normalize loss

            scaler.scale(loss).backward()

            if (i + 1) % ACCUMULATION_STEPS == 0:
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()

            total_loss += loss.item() * ACCUMULATION_STEPS
            loop.set_description(f"Epoch {epoch+1}")
            loop.set_postfix(loss=loss.item() * ACCUMULATION_STEPS)

        print(f"Avg Loss: {total_loss / len(train_loader)}")

    # --- 7. EVALUATION ON TRICKY CASES ---
    print("\nEvaluating on Tricky Adversarial Cases...")
    model.eval()

    def predict(text):
        text = clean_text(text) # Clean input before predicting
        encoding = tokenizer.encode_plus(
            text, add_special_tokens=True, max_length=MAX_LEN,
            return_token_type_ids=False, padding='max_length',
            truncation=True, return_attention_mask=True, return_tensors='pt',
        )
        input_ids = encoding['input_ids'].to(DEVICE)
        mask = encoding['attention_mask'].to(DEVICE)
        with torch.no_grad():
            outputs = model(input_ids, attention_mask=mask)
            probs = torch.nn.functional.softmax(outputs.logits, dim=1)
        conf, idx = torch.max(probs, dim=1)
        return classes[idx.item()], conf.item()

    test_cases = [
        "Marketing Intern assisting with social media scheduling.", # Should be Internship
        "Virtual Intern: Participated in a 4-week remote simulation.", # Should be Internship
        "Engineering Lead responsible for mentoring 3 summer interns.", # Should be Full-time
        "Full-time Implementation Consultant working exclusively for Oracle.", # Should be Full-time
        "Internal Strategy Consultant employed permanently by the firm.", # Should be Full-time
        "Freelance Web Developer building WordPress sites.", # Should be Freelance
        "Temporary Staff handling seasonal overflow.", # Should be Freelance (or Full-time depending on definition, but usually Freelance/Contract)
        "Acting Interim Lead for the design team during a summer placement." # Should be Full-time
    ]

    print(f"{'PREDICTION':<15} | {'CONF.':<8} | {'TEXT'}")
    print("="*80)
    for text in test_cases:
        pred, conf = predict(text)
        color = "🟢" if conf > 0.8 else "🔴"
        print(f"{pred.upper():<15} | {conf*100:.1f}% {color} | {text}")

    # Save
    SAVE_PATH = "./robust_bert_model"
    if not os.path.exists(SAVE_PATH): os.makedirs(SAVE_PATH)
    model.save_pretrained(SAVE_PATH)
    tokenizer.save_pretrained(SAVE_PATH)
    np.save(os.path.join(SAVE_PATH, 'classes.npy'), classes)
    print(f"\nModel saved to {SAVE_PATH}")

Loading and cleaning data...
Training Data Size after injection: 34088

Initializing BERT with high Dropout (to prevent overfitting)...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\charls\AppData\Local\Temp\ipykernel_21444\4213858408.py:165: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()



Starting Training...


  0%|          | 0/7670 [00:00<?, ?it/s]C:\Users\charls\AppData\Local\Temp\ipykernel_21444\4213858408.py:182: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 1:   2%|▏         | 127/7670 [00:10<10:25, 12.06it/s, loss=0.14]  


KeyboardInterrupt: 

In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import re
import random
import gc
import os
from tqdm import tqdm

# --- CONFIGURATION ---
INPUT_FILE = "robust_experience_training_data_unbalanced.csv"
MAX_LEN = 128
BATCH_SIZE = 4
ACCUMULATION_STEPS = 8  # Effective Batch Size = 32
EPOCHS = 6              
LR = 2e-5
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Running on: {DEVICE}")

Running on: cuda


In [3]:
# --- ROBUST TEXT CLEANING ---
def clean_text(text):
    text = str(text).lower()
    # Remove dates (e.g., Jan 2020, 2019-2022) to prevent date-overfitting
    text = re.sub(r'\b(jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)[a-z]*\.? \d{4}', '', text)
    text = re.sub(r'\d{4}\s?-\s?(present|current|\d{4})', '', text)
    # Remove locations 
    text = re.sub(r'[a-z]+, [a-z]{2} \|', '', text) 
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# --- GENERATE INTELLIGENT ADVERSARIAL DATA ---
def generate_adversarial_data():
    data = []

    # A. TEACH: "Consultant" can be Full-time
    ft_consultant_templates = [
        "Full-time Implementation Consultant at Oracle. I work permanently for the firm.",
        "Senior Strategy Consultant (Internal Employee). Not a contractor.",
        "Staff Consultant. Led the client project as a full-time employee.",
        "Permanent Solutions Consultant. Salaried role with benefits.",
        "Principal Consultant. I was employed full-time to manage key accounts."
    ]
    for _ in range(200): 
        data.append({'text': random.choice(ft_consultant_templates), 'label': 'Full-time'})

    # B. TEACH: "Managing Interns" is Full-time, not Internship
    ft_manager_templates = [
        "Engineering Lead. My responsibilities included mentoring summer interns.",
        "Senior Manager. I managed a team of 3 interns and 2 juniors.",
        "Program Coordinator. I organized the internship orientation.",
        "Director of Operations. Oversaw the summer analyst internship program.",
        "Team Lead. Conducted code reviews for the intern team."
    ]
    for _ in range(200):
        data.append({'text': random.choice(ft_manager_templates), 'label': 'Full-time'})

    # C. TEACH: "Intern" means you are learning, not working full-time
    intern_templates = [
        "Marketing Intern. Assisted the manager with social media.",
        "Summer Analyst. Rotational program in finance and risk.",
        "Engineering Intern. Shadowed senior developers and fixed bugs.",
        "Co-op Student. 4-month placement during university.",
        "Virtual Intern. Completed simulated tasks for the company."
    ]
    for _ in range(200):
        data.append({'text': random.choice(intern_templates), 'label': 'Internship'})

    # D. TEACH: "Contractor/Freelance" vs Full-time
    freelance_templates = [
        "Independent Contractor. Worked on a per-project basis.",
        "Freelance Consultant. Advised multiple clients simultaneously.",
        "Self-employed Developer. Built websites for local businesses.",
        "Gig Worker. Completed tasks via a mobile platform.",
        "Temporary Staff. 3-month contract to cover seasonal demand."
    ]
    for _ in range(200):
        data.append({'text': random.choice(freelance_templates), 'label': 'Freelance'})

    return pd.DataFrame(data)

In [4]:
if not os.path.exists(INPUT_FILE):
    print(f"Error: {INPUT_FILE} not found. Please ensure your base CSV is present.")
else:
    print("Loading and cleaning data...")
    df = pd.read_csv(INPUT_FILE)
    
    # 1. Clean original data
    df['text'] = df['text'].apply(clean_text)
    
    # 2. Inject "Smart Poison"
    df_poison = generate_adversarial_data()
    df = pd.concat([df, df_poison], ignore_index=True)
    
    # 3. Shuffle
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)

    print(f"Training Data Size after injection: {len(df)}")

    # 4. Encode Labels
    le = LabelEncoder()
    df['label_encoded'] = le.fit_transform(df['label'])
    classes = le.classes_

    # 5. Split
    X_train, X_test, y_train, y_test = train_test_split(
        df['text'].values, df['label_encoded'].values, test_size=0.1, random_state=42, stratify=df['label_encoded']
    )
    
    print("Data prepared successfully.")

Loading and cleaning data...
Training Data Size after injection: 34088
Data prepared successfully.


In [5]:
if not os.path.exists(INPUT_FILE):
    print(f"Error: {INPUT_FILE} not found. Please ensure your base CSV is present.")
else:
    print("Loading and cleaning data...")
    df = pd.read_csv(INPUT_FILE)
    
    # 1. Clean original data
    df['text'] = df['text'].apply(clean_text)
    
    # 2. Inject "Smart Poison"
    df_poison = generate_adversarial_data()
    df = pd.concat([df, df_poison], ignore_index=True)
    
    # 3. Shuffle
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)

    print(f"Training Data Size after injection: {len(df)}")

    # 4. Encode Labels
    le = LabelEncoder()
    df['label_encoded'] = le.fit_transform(df['label'])
    classes = le.classes_

    # 5. Split
    X_train, X_test, y_train, y_test = train_test_split(
        df['text'].values, df['label_encoded'].values, test_size=0.1, random_state=42, stratify=df['label_encoded']
    )
    
    print("Data prepared successfully.")

Loading and cleaning data...
Training Data Size after injection: 34088
Data prepared successfully.


In [6]:
class ResumeDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]
        encoding = self.tokenizer.encode_plus(
            text, add_special_tokens=True, max_length=self.max_len,
            return_token_type_ids=False, padding='max_length',
            truncation=True, return_attention_mask=True, return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_dataset = ResumeDataset(X_train, y_train, tokenizer, MAX_LEN)
test_dataset = ResumeDataset(X_test, y_test, tokenizer, MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)
print("Dataloaders ready.")

Dataloaders ready.


In [ ]:
print("Initializing BERT with high Dropout...")
config = BertConfig.from_pretrained(
    'bert-base-uncased', 
    num_labels=len(classes),
    hidden_dropout_prob=0.3,          # High dropout to prevent overfitting
    attention_probs_dropout_prob=0.3  
)
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', config=config)
model = model.to(DEVICE)

optimizer = AdamW(model.parameters(), lr=LR)
# scaler = torch.cuda.amp.GradScaler() # For mixed precision training
# Old way (what you have):
# scaler = torch.cuda.amp.GradScaler()

# New way (removes warning):
scaler = torch.amp.GradScaler('cuda')

print("Model initialized.")

Initializing BERT with high Dropout...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model initialized.


In [11]:
print("Starting Training...")

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    loop = tqdm(train_loader, leave=True)
    
    optimizer.zero_grad()
    
    for i, batch in enumerate(loop):
        input_ids = batch['input_ids'].to(DEVICE)
        mask = batch['attention_mask'].to(DEVICE)
        labels = batch['labels'].to(DEVICE)

        with torch.cuda.amp.autocast():
            outputs = model(input_ids, attention_mask=mask, labels=labels)
            loss = outputs.loss / ACCUMULATION_STEPS # Normalize loss

        scaler.scale(loss).backward()

        if (i + 1) % ACCUMULATION_STEPS == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

        total_loss += loss.item() * ACCUMULATION_STEPS
        loop.set_description(f"Epoch {epoch+1}")
        loop.set_postfix(loss=loss.item() * ACCUMULATION_STEPS)

    print(f"Avg Loss: {total_loss / len(train_loader)}")

Starting Training...


  0%|          | 0/7670 [00:00<?, ?it/s]

C:\Users\charls\AppData\Local\Temp\ipykernel_21444\1318387829.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 1:  14%|█▍        | 1069/7670 [10:25<1:04:20,  1.71it/s, loss=0.0207]


KeyboardInterrupt: 

In [ ]:
print("\nEvaluating on Tricky Adversarial Cases...")
model.eval()

def predict(text):
    text = clean_text(text) # Use the same cleaning function
    encoding = tokenizer.encode_plus(
        text, add_special_tokens=True, max_length=MAX_LEN,
        return_token_type_ids=False, padding='max_length',
        truncation=True, return_attention_mask=True, return_tensors='pt',
    )
    input_ids = encoding['input_ids'].to(DEVICE)
    mask = encoding['attention_mask'].to(DEVICE)
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=mask)
        probs = torch.nn.functional.softmax(outputs.logits, dim=1)
    conf, idx = torch.max(probs, dim=1)
    return classes[idx.item()], conf.item()

test_cases = [
    "Marketing Intern assisting with social media scheduling.", 
    "Virtual Intern: Participated in a 4-week remote simulation.", 
    "Engineering Lead responsible for mentoring 3 summer interns.", 
    "Full-time Implementation Consultant working exclusively for Oracle.", 
    "Internal Strategy Consultant employed permanently by the firm.", 
    "Freelance Web Developer building WordPress sites.", 
    "Temporary Staff handling seasonal overflow.", 
    "Acting Interim Lead for the design team during a summer placement." 
]

print(f"{'PREDICTION':<15} | {'CONF.':<8} | {'TEXT'}")
print("="*80)
for text in test_cases:
    pred, conf = predict(text)
    color = "🟢" if conf > 0.8 else "🔴"
    print(f"{pred.upper():<15} | {conf*100:.1f}% {color} | {text}")

In [ ]:
SAVE_PATH = "./robust_bert_model"
if not os.path.exists(SAVE_PATH): os.makedirs(SAVE_PATH)
model.save_pretrained(SAVE_PATH)
tokenizer.save_pretrained(SAVE_PATH)
np.save(os.path.join(SAVE_PATH, 'classes.npy'), classes)
print(f"\nModel saved to {SAVE_PATH}")